### Patterns for Cleaner Python

#### Assertions 

Author claims that assertions are a seriously under-appreciated language feature which allows for better error-detection and debugging. 

Imagine that we're building some functionality to apply a discount code to a product:

In [ ]:
shoes = {'name': 'Fancy Shoes', 'price': 14900}

def apply_discount(product, discount):
    price = int(product['price'] * (1.0 - discount))
    assert 0 <= price <= product['price']
    return price 

apply_discount(shoes, .25)

Note first that these shoes aren't $14,900 dollars, but instead 149.00 represented without a decimal. This avoids the trouble rounding and dealing with floats. 

For a valid discount amount, we have no trouble. But for an invalid amount: 

In [ ]:
apply_discount(shoes, 2.0)

Our invalid argument throws an AssertionError, with a pointer to the line that is causing the problems. But what advantages does this have over regular exceptions? 

The author makes the case that assertions are meant as internal sanity checks in the code. Regular exceptions work beautifully for glaring errors such as passing an incorrect data type or referencing a non-existent file, but assertions make finding more subtle bugs a great deal easier. 

However, he goes on to note that assertions aren't intended to handle run-time errors. 

Our comprehension will be aided by an exposition on the internals of assertion implementation. According to the python documentation this is essentially what an assertion looks like under-the-hood:

`assert_stmt ::= "assert" expression1 ["," expression2]`

This boils down to: assert the first expression, and if that doesn't work, display the optional error message in expression2. 

The interpreter translates our written assert statements into something like this:

`if __debug__: 
    if not expression1:
        raise AssertionError(expression2)
`

Notice that the first thing this code does is check for the `__debug__` global variable, which is `True` under normal circumstances but not if optimizations are requested. Also, the optional expression2 gives us quite a lot of flexibility in bug checking. We can give ourselves messages, or include our email address so that users can send us information related to our code's failure. 

There are two major caveats to using assertions. The first involves the possible introduction of security risks into code if assertions are used to check whether or not an argument contains an incorrect or unexpected value. Assertions can be globally disabled in both the command line and CPython, causing all assertions to be compiled away at runtime. 

To see why this matters, take a look at the following code:

In [ ]:
def delete_product(prod_id, user):
    assert user.is_admin(), "Sorry, you have to be an admin."
    assert store.has_product(prod_id), "We don't recognize that ID."
    store.get_product(prod_id).delete()

In this case, if the assert statements are turned into null operations neither of them run. This means that the check for the user's admin status doesn't run, and anyone can delete products, and that we can 'delete' invalid products, which could set us up for attacks or other nastiness later. 

The solution here is never to use assertions to validate data. In this example it'd be far safer to do the following:

In [ ]:
def delete_product(prod_id, user):
    if not user.is_admin():
        raise AuthError("Sorry, you have to be an admin.")
    if not store.has_product(prod_id):
        raise ValueError("We don't recognize that ID.")
    store.get_product(prod_id).delete()

The second caveat involves the danger of writing assertions which simply never fail. An example hinges on the fact that non-empty tuples are truthy in Python, and therefore if a tuple is passed to an assert statement, it will always evaluate to true. 

This:

`assert(1==2, 'This should fail')`

Actually never fails. 

Another example:

`
assert(
    counter == 10,
    "It should've counted all the items"
)
`

This looks harmless enough, but doesn't work for the exact same reason. For this reason it's good to use a 'code linter' (I don't know what this is yet), and to ensure that tests in a test suite actually do fail before moving on. 

#### Comma placement 

Another technique we can use to avoid headaches is to train ourselves to use commas at the end of every line of code. Lists are often defined like this:

In [ ]:
names = ['Alice', 'Bob', 'Dilbert']

The issue with this is that VCSs tend to be line-based, and do poorly when multiple changes are made to a single line. So if we spread this list out, we'll have an easier time tracking changes with something like Github down the road:

In [ ]:
names = [
        'Alice',
        'Bob',
        'Dilbert'
]

Better, but if we have to add an item at the end or delete the item at the end we'll have to remember to manually update the comma information on pain of being bitten by literal string concatenation:

In [ ]:
names = [
    'Alice',
    'Bob',
    'Dilbert'
    'Jane'
]

In [ ]:
names

What happened here is that Python automatically concatenated the last two items because I forgot to add a comma after 'Dilbert'. All of this goes away if we just add a comma after every line, even the last one. This isn't a problem for Python and will help avoid all the tribulations detailed above:

In [ ]:
names = [
    'Alice',
    'Bob',
    'Dilbert',
    'Jane',
]

In [ ]:
names

Note that this also works on dictionaries and sets. 

#### Context Managers and the with Statement 

The with statement 'helps simplify some common resource management patterns by abstracting their functionality and allowing them to be factored out and reused.' Here we have a pretty common operation, opening a file with an alias and then doing stuff with it:

`
with open('hello.txt', 'w') as f:
    f.write('hello, world')
`

Opening files this way, with the `with` statement, is considered best practices because Python will automatically close it when execution leaves the with statement's context. Internally this code parses roughly to: 

`
f = open('hello.txt', 'w')
try:
    f.write('hello, world')
finally:
    f.close()
`

The advantage here is that Python automatically closes the filename. If it didn't, an exception thrown during `.write()` might cause our program to leak a file descriptor. `with` has made resource management much easier because we don't have to write out the `try`/`finally` loop, and it's nearly impossible for us to forget to close files and otherwise release a resource when we're done with it. 

`with` is a kind of context manager which, more broadly, can be implemented in any class or object using `__enter__` and `__exit__`. The `open` context manager might look something like this:

In [ ]:
class ManagedFile:
    def __init__(self, name):
        self.name = name
    
    def __enter__(self):
        self.file = open(self.name, 'w')
        return self.file
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if self.file:
            self.file.close()

This means that our `ManagedFile` class will work with `with` now, in the manner to which we've grown accustomed:

`
with ManagedFile('hello.txt') as f:
    f.write('hello, world')
    f.write('Goodbye')
`

The way we've written the class means that Python will call `__enter__` when program execution enters the `with` statement's context and will call `__exit__` when it leaves, which frees up the resource.

The Python standard library comes with a powerful set of context management tools, via the `contextlib` module. To take one example, we can utilize `contextmanage` as a decorator to create a 'factory function' powered by a generator that automatically supports the `with` statement. 

Behold:

In [ ]:
from contextlib import contextmanager

@contextmanager
def managed_file(name):
    try:
        f = open(name, 'w')
        yield f 
    finally: 
        f.close()

We could then call it as we've done everything else:
`
with managed_file('hello.txt') as f:
    f.write('hello, world')
    f.write('bye now')
`
What exactly is happening here? This generator-based function essentially opens the resource (a file in this case), then yields it for the function caller's use, before going on to finish clean up and closing. 

It is basically equivalent to the class approach defined above. 

#### Context managers and APIs. 

Context managers can be used to create APIs. Observe:

In [19]:
class Indenter:
    def __init__(self):
        self.level = 0
    
    def __enter__(self):
        self.level += 1
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.level -= 1
        
    def print(self, text):
        print('  ' * self.level + text)

with Indenter() as indent:
    indent.print('Hi!')
    with indent:
        indent.print("Guten Tag!")
        with indent:
            indent.print("Hola!")
    indent.print("Yo")

  Hi!
    Guten Tag!
      Hola!
  Yo


This is redolent of a domain-specific language (DSL) specifically for text indentation. 

#### Underscores, Dunders, and More 

There are a number of ways in which underscores are used in variable and method names, some of which are just conventions meant to signal some functionality to programmers, some of which are actually important to the interpreter. 

Here are the big 5:

In [20]:
# _var          # single leading under
# var_          # single trailing under
# __var         # double leading under 
# __var__       # double surrounding under 
# _             # single under 

A single prefix like `_var` is meant to signal that a variable is for internal use. It's a convention only, the intepreter doesn't care. 

In [26]:
class Test:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        
t = Test()
print(t.foo,',', t._bar)

11 , 23


As can be seen here, Python doesn't stop us from grabbing a `_var`, but by convention you aren't supposed to. 

The case is a bit more complicated when importing functions whose names containing leading underscores. Say we have a module we've defined:

In [27]:
# my_module:

def external_func():
    return 23

def _internal_func():
    return 42

If we then go on to use wildcard syntax to import my_module:

`from my_module import *`

The internal function won't be imported (though it will be if a standard `import my_module` is used instead; this is one reason why standard imports are superior to wildcard imports, because the latter make it unclear what's lined up in the namespace. PEP8 actually discourages wildcard imports). 

By convention a trailing underscore is mostly used to prevent conflicts with reserved keywords. So if you find yourself in a situation where the best name for a variable is `class`, you're going to have a bad time; that keyword is reserved. But you know what isn't reserved? `class_`.

The Python interpreter *does* care about preceding dunders, however, like `__var`. In what is usually called *name mangling*, the interpreter will actually *change* the names of dunder-named variables to prevent conflicts with subclasses which might have attributes whose names are similar. 

Observe:

In [30]:
class Test2:
    def __init__(self):
        self.foo = 11
        self._bar = 23
        self.__baz = 42
        
# Let's instantiate
t = Test2()

# And use .dir() to figure out what attributes are available
dir(t)

['_Test2__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

Notice that in the attribute list `foo` and `_bar` are left unchanged, while `__baz` has been *name mangled* into `Test__baz`. Why do this? So that the name isn't overridden in subclasses. 

An example:

In [32]:
class ExtendedTest(Test2):
    def __init__(self):
        self.foo = 'overridden'
        self._bar = 'overridden'
        self.__baz = 'overridden'
        
t2 = ExtendedTest()

t2.foo, t2._bar

('overridden', 'overridden')

In [33]:
# However...
t2.__baz

AttributeError: 'ExtendedTest' object has no attribute '__baz'

In [34]:
# Where is the __baz attribute? 
dir(t2)

['_ExtendedTest__baz',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bar',
 'foo']

As before, Python has name mangled this variable because of its preceding double underscore. If we want to get it out, we have to use the new interpreter-given name:

In [36]:
t2._ExtendedTest__baz

'overridden'

It is possible, however, to define a method within a class which *can* successfully get the relevant attribute out, even when said attribute can't be accessed directly.

In [37]:
class MangledTest:
    def __init__(self):
        self.__mangled = 'hello'
        
    def get_mangled(self):
        return self.__mangled
    
mt = MangledTest()

In [38]:
mt.__mangled

AttributeError: 'MangledTest' object has no attribute '__mangled'

In [39]:
mt.get_mangled()

'hello'

And this applies to methods as well.

In [40]:
class MangledMethod:
    def __method(self):
        return 42
    def call_it(self):
        return self.__method()

In [41]:
mm = MangledMethod()
mm.__method()

AttributeError: 'MangledMethod' object has no attribute '__method'

In [42]:
mm.call_it()

42

Name mangling does not apply to attribute or method names with both leading and trailing double underscores. These so-called *magic methods* are reserved for special use cases in Python. 

The single underscore _ is a naming convention generally used to denote a temporary value about which no one really cares. We might use it in tuple unpacking when we're discarding some of the unpacked values. 

In [43]:
car = ('red', 'auto', 12, 2800)
color, _, _, mileage = car

In [47]:
color, mileage

('red', 2800)

#### String Formatting

In [51]:
# There are four major ways to do string formatting. The first is called the 'old style'

name = 'Bob'
errno = 50159747054

# With Python's % operator we can do quick positional string formatting.

print("Hey %s, I'm afraid there is a 0x%x error" % (name, errno))

Hey Bob, I'm afraid there is a 0xbadc0ffee error


The % operator takes but one argument, which can be an n-ary tuple, or we can accomplish the same thing with a dictionary mapping.

In [52]:
print("Hey %(name)s, there is a 0x%(errno)x error" % {'name': name, 'errno': errno})

Hey Bob, there is a 0xbadc0ffee error


New style string formatting is more regular and requires calling a special .format() method on a string object. We can do the same things as before, it just necessitates a few syntactical changes.

In [54]:
print("Hello {name}, there's a 0x{errno:x} error!".format(name=name, errno=errno))

Hello Bob, there's a 0xbadc0ffee error!


Starting in Python 3.6 we can now make use of formatted string literals to do literal string interpolation directly in the string itself. 

In [55]:
a,b = 5,10
print(f'Five plus ten is {a + b} and not {2 * (a + b)}')

Five plus ten is 15 and not 30


This becomes even more powerful when we put it into a function. 

In [58]:
def greet(name, question):
    return f"Hello {name}, how's it {question}"

greet('bob','feeling')

"Hello bob, how's it feeling"

A final method relies on the Template function from Python's `string` module. Though it isn't nearly as flexible as what's covered above, it might serve in certain straightforward cases. 

Template strings aren't a core language feature, and they can't do things like take a format specifier which will allow us to convert our integer error into a hexadecimal string. We have to do that manually. 

Given these shortcomings, when might we use such a technique? One use case is in fetching strings from users. Here, the greater simplicity of Template strings makes them a safer option. 

In [60]:
from string import Template
t = Template('Hey, $name')
t.substitute(name=name)

'Hey, Bob'

It is not impossible for there to arise situations in which a user can access secret keys by passing in a format string

In [62]:
SECRET = 'You should not be here'

class Error:
    def __init__(self):
        pass

err = Error()
user_input = '{error.__init__.__globals__[SECRET]}'

user_input.format(error=err)

'You should not be here'

Template strings close down this attack vector. 

Given this embarrassment of riches, how should we approach string formatting? The author says that if our code deals in user-supplied strings, we should use Template strings, if we're in Python 3.6+ we should use literal string interpolation, otherwise use the 'new style' (with `.format()`). 

### Effective Functions

Python functions are as versatile as they are powerful. They can be stored in variables and data structures, passed as arguments to other functions, and even returned as values in functions. The examples throughout the chapter utilize this basic function:

In [63]:
def yell(text):
    return text.upper() + "!"

print(yell("Hello"))

HELLO!


Python is object-oriented because everything in it is an object. Strings, lists, variables, modules, and functions are all objects. 

This means that the function `yell` can be assigned to a variable, as can any other object.  

In [66]:
bark = yell # Note that we're not calling it. If we do that throws an error.
bark('woof')

'WOOF!'

A function is not it's name. In the following we delete the function name `yell` but then still call it because we've pointed another name at the underlying functionality.

In [68]:
del yell
yell('hello')

NameError: name 'yell' is not defined

In [69]:
bark('hello')

'HELLO!'

In [70]:
# But, Python automatically attaches a string identifier to every function as an aid in 
# debugging. It can be accessed as the .__name__ attribute. 

bark.__name__

'yell'

Functions can be stored inside data structures like lists, but be sure not to call them!

In [72]:
funcs = [bark, str.capitalize, str.lower]

# With this in place we can call the functions as we would in any other context

for f in funcs:
    print(f, f('Hello'))

<function yell at 0x10707d1e0> HELLO!
<method 'capitalize' of 'str' objects> Hello
<method 'lower' of 'str' objects> hello


In [74]:
# This list allows us to use 'disembodied functions', which involve looking up and then calling
# a function. 

funcs[0]('hey')

'HEY!'

In [77]:
# And because functions are objects they can be passed to other functions.

def greet(func, string):
    greeting = func(string)
    return greeting

print(greet(bark, 'Hi'))

HI!


Passing functions to other functions gives us the option of focusing on behavior. The `greet` function above can do all kinds of things depending on the function which we pass to it. 

This is an example of a higher-order function, the classic example of which is the `.map()` function. `.map()` takes a function and an iterable and applies the function to each iterable. 

In [78]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

We can also define *nested* or *inner* functions inside other functions. This means that each time the outer function is called it defines the inner function, which has no existence beyond the outer functione

In [79]:
def speak(text):
    def whisper(t):
        return t.lower() + '...'
    return whisper(text)

print(speak('HELLO WORLD'))

hello world...


This leaves us unable to access the inner function, however, because it is resurrected each time the outer function calls. Should we truly desire in our hearts to access this function, however, we could always return it to the caller. The returned function isn't actually called, mind you, it is just returned. 

In [88]:
def get_speak(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > .5:
        return yell
    else:
        return whisper
    
print(get_speak(.8)('hello'))

HELLO!


What this ultimately means is that Python's adherence to OO principles means that we can abstract away the function syntax and semantics and instead traffic in *behaviors*, both as inputs and outputs. 

#### Functions can capture local state  

In addition to the fact that functions can take and return other functions, inner functions can capture and carry some of their parent function's state. In the above `get_speak` function the inner functions took a parameter. If we re-write them so that they don't, we can have inner functions called `lexical closures` which remember the values passed to the parent function:

In [82]:
def get_speak(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!'
    if volume < .5:
        return whisper
    else:
        return yell

get_speak('hello', .7)()

'HELLO!'

Here is the same idea implemented in a so-called *factory function*

In [90]:
def make_adder(n):
    def add(x):
        return x + n
    return add 

plus_3 = make_adder(3)
plus_5 = make_adder(5)

plus_5(5)

10

#### Objects Can Behave Like Functions 

All functions are objects but not all objects are functions. However, it is possible to use the `__call__` dunder method to make objects which behave like functions

In [92]:
class Adder:
    def __init__(self, n):
        self.n = n
    
    def __call__(self, x):
        return self.n + x
    
plus_10 = Adder(10)
print(plus_10(5))

15


What's actually happening here is that when the object instance in `plus_10` is 'called' like a function the interpreter attempts to execute the `__call__` method. If we'd like to verify whether or not a given object is callable we can use the built-in `callable` method, which tells us whether or not whatever we pass in appears to be callable. 

In [94]:
    print(callable('probably not callable'))

False


#### Lambdas are single-expression functions 

Lambda is python's shortcut way of defining small, anonymous functions, like this one:

In [1]:
add = lambda x, y: x + y
add(2,3)

5

This actually let's us define and immediately call functions inline with *function expressions*.

In [2]:
(lambda x, y: x * y)(3, 5)

15

Because lambdas are restricted to a single expression they can't use statements like `return`, though because lambda automatically execute and return values there is a kind of implicit return statement. 

Lambdas can be used anywhere a function object is required, but a common use case is for sorting iterables by some alternate key:

In [3]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [8]:
sorted(range(-5,6), key=lambda x: x**2)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

#### The Power of Decorators 

Decorators allow for us to non-permanently change the behavior of a callable like a class or function. Common use cases are logging behavior, timing, caching, enforcing authentication. To understand decorators we need to review two facts about first-class functions in Python: first, functions can be assigned to variables and returned from other functions; second, they can be defined inside of other functions. 

A decorator is a callable that takes another callable as input and returns a callable. A null decorator might look like this:

In [9]:
def null_decorator(func):
    return func

Anytime we wanted to use this decorator now we'd append `@null_decorator` just above the function we want to decorate. 

Here is a more complicated decorator:

In [22]:
def uppercase(func):
    def wrapper():
        original_result = func()
        modified_result = original_result.upper()
        return modified_result
    return wrapper

@uppercase
def greet():
    return 'Hello!'

greet()

'HELLO!'

While our `null_decorator` simply returned the original function, the more complicated `uppercase` must wrap input functions in a closure in order to influence the future behavior of the input function. Behavior is modified through this wrapper so that the original function isn't modified.

#### Applying Multiple Decorators to a Function 

In [24]:
def strong(func):
    def wrapper():
        return '<strong>' + func() + '</strong>'
    return wrapper 

def emphasis(func):
    def wrapper():
        return '<em>' + func() + '</em>'
    return wrapper 

# The can both be applied simultaneously to greet.

@strong
@emphasis

def greet():
    return 'Hello'

greet()

'<strong><em>Hello</em></strong>'

Stacked decorators are applied from the bottom to the top, and the above example is roughly equivalent to `decorated_greet = strong(emphasis(greet))`.

So far, though, we haven't decorated functions which accept arguments. These can be tricky, and they're why we use `*args` and `**kwargs`. For example:

In [25]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper 

There are two things happening here. First, the closure `wrapper` function is using `*` and `**` to collect arbitrarily many arguments passed to `func`, then it forwards them to the input function using the `*` and `**` argument unpacking operations. 

In [29]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() '
              f'with {args}, {kwargs}')
        
        original_result = func(*args, **kwargs)
        
        print(f'TRACE: {func.__name__} '
              f'returned {original_result!r}')
        
        return original_result
    return wrapper 

@trace
def say(name, line):
    return f'{name}: {line}'

say('Jane', 'Hello World')

TRACE: calling say() with ('Jane', 'Hello World'), {}
TRACE: say returned 'Jane: Hello World'


'Jane: Hello World'

One downside of this approach is that some of a function's metadata is masked by the wrapper's metadata once it has been decorated, with debugging consequently becoming harder. A workaround is to import `functools.wraps` and use it as a decorator inside the decorator function, where it copies over the input function's metadata and keeps that available. 

In [30]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper 

#### Fun With `*args` and `**kwargs` 

`*args` and `**kwargs` allow for the creation of more flexible APIs for modules and classes by allowing optional arguments in function definitions. Because `*args` begins with an `*`, it collects any optional arguments passed to it into a tuple. Similarly because `**kwargs` begins with an `**` it will collect keyword arguments in a dictionary. 

In [31]:
def foo(required, *args, **kwargs):
    print(required)
    if args:
        print(args)
    if kwargs:
        print(kwargs)

In [32]:
foo() # --> throws an error because we haven't passed in the correct argument. 

TypeError: foo() missing 1 required positional argument: 'required'

In [33]:
foo('hello')

hello


In [34]:
foo('hello', 1, 2, 3)

hello
(1, 2, 3)


In [35]:
foo('hello', 1,2,3, key='value', key2 = 999)

hello
(1, 2, 3)
{'key': 'value', 'key2': 999}


By making use of the `*` and `**` operators we can actually forward optional/keyword arguments to functions and modify the arguments as we go along. 

Observe:

In [36]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', )
    bar(x, *new_args, **kwargs)

This has applications in writing wrappers and dealing with subclasses.

In [38]:
class Car:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage

class AlwaysBlueCar(Car):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.color = 'blue'

AlwaysBlueCar('green', 413).color

'blue'

The way these are written, the child class `AlwaysBlueCar` inherits its arguments from its parent class and then overrides one of its internal attributes. In theory this child class might still perform just fine if its parent class changed. Unfortunately, we don't know what arguments it wants without looking up the definition of its parent class. 

`*args` and `**kwargs` are also useful in writing functions like decorators, where we'll frequently be in a situation in which we want our decorated function to be able to take arbitrarily many input arguments. 

In [41]:
def trace(f):
    @functools.wraps(f)
    def decorated_function(*args, **kwargs):
        print(f, args, kwargs)
        result = f(*args, **kwargs) # passing those optionals in to the input function 
        print(result)
    return decorated_function

@trace
def greet(greeting, name):
    return f'{greeting}, {name}'

greet('sup', 'Joe')

<function greet at 0x110e18048> ('sup', 'Joe') {}
sup, Joe


### Function Argument Unpacking 

Python let's us unpack function arguments from sequences and dictionaries with the `*` and `**` syntax. Let's define a simple function to use as an example 

In [44]:
def print_vector(x, y, z):
    print('<%s, %s, %s>' % (x,y,z))

print_vector(1,2,3)

<1, 2, 3>


Unfortunately we're constrained a bit here by which data structure we use to represent our vector. If we want to store variables in a tuple then we'll be forced to use indexing to get them out of the tuple when it comes time to pass in arguments to `print_vector`

In [46]:
tuple_vec = (1,0,1)
print_vector(tuple_vec[0], tuple_vec[1], tuple_vec[2])

<1, 0, 1>


Who has time for this? Luckily for us we can leverage the magic of tuple unpacking 

In [47]:
print_vector(*tuple_vec)

<1, 0, 1>


It even works for generators

In [49]:
genexpr = (x * x for x in range(3))
print_vector(*genexpr)

<0, 1, 4>


Similarly, `**` let's us unpack a dictionary, which would be useful if we had stored our vector coordinates in a dictionary for some unconceivable reason

In [50]:
vec_dict = {'y': 0, 'z': 1, 'x': 1}
print_vector(**vec_dict)

<1, 0, 1>


#### Nothing to Return Here 

Python implicitly returns `None` for any function which lacks an explicit `return` statement. Author's rule of thumb is to leave out the return statement if the function doesn't have or doesn't need one, meaning that the function will just return `None`. This isn't uncommon, `print()` returns `None`, and is in fact used exclusively for its side effect. 

What are the tradeoffs? On the one hand, letting returns be implicit makes code more concise and (arguably) 'prettier'. On the other, the implicitly-returned `None` likely isn't obvious to everyone (it certainly wasn't to me when I first encountered it), and from a maintenance standpoint anything surprising is likely to be a liable down the road. 

### Chapter 4, Classes & OOP

#### Object Comparisons: 'is' versus '==' 

Equality and identity aren't the same thing. Two lists can be equal, having the same objects, without actually pointing to the same underlying data structure in memory. 

In [51]:
a = b = [1,3,5]

In [52]:
a == b

True

In [53]:
a is b

True

In [54]:
a = [4,5,6] 
b = [4,5,6] 

a is b, a == b

(False, True)

#### String Conversion

If we define a class and then try to print it for the purposes of inspecting it, we get a class name and id, corresponding to the memory address in CPython. 

In [55]:
class Car: 
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage
        
my_car = Car('red', 112233)
print(my_car)

The best course of action here is to add the dunder `__str__` or `__repr__` methods to our class, which afford us a fair bit of control over how objects are transformed into strings in various scenarious. 

In [59]:
class betterCar:
    def __init__(self, color, mileage):
        self.color = color 
        self.mileage = mileage 
        
    def __str__(self):
        return f'A {self.color} car'
    
my_better_car = betterCar('green', 112233)
print(my_better_car)

A green car


How is `__str__` different from `__repr__`? Both control how objects are converted into strings, but are useful in different situations. 

In [61]:
class thirdCar:
    def __init__(self, color, mileage):
        self.color = color
        self.mileage = mileage 
        
    def __repr__(self):
        return '__repr__ for thirdCar'
    
    def __str__(self):
        return '__str__ for thirdCar'
    
my_third_car = thirdCar('red', 112233)
print(my_third_car)

__str__ for thirdCar


In [62]:
my_third_car

__repr__ for thirdCar

What this means is that when I toss an object into the console for inspection what's returned is that object's `__repr__`. By using the relevant dunder method I can exercise some influence over how that process works. Python also has built-in `str()` and `repr()` methods which basically do the same thing, and it's best practice to use those. 

The `__str__` method can be thought of as generally returning the kind of human-readable string we might display to a user, while the `__repr__` method instead returns a string that a programmer might use for debugging. 

If `__repr__` isn't added to a class then Python defaults to it when looking for `__str__`, with a correspondingly ugly output. It's therefore a good idea to add `__repr__` methods to class definitions.  

This is nice and concise:

In [64]:
def __repr__(self):
    return (f'{self.__class__.__name__}('f'{self.color!r}, {self.mileage!r})')

# The !r is a conversion flag which tells this function to use the repr's for color and mileage.


#### Defining Your Own Exception Classes

Custom-built exception classes can be quite useful for debugging and maintaining code. It takes a bit of engineering to get the maximum amount of usefulness out of them. Observe:

In [66]:
def validate(name):
    if len(name) < 10:
        raise ValueError
        
validate('Trent')

ValueError: 

This stack trace isn't super useful. If we instead define a class which handles this error, we get a kind of self-documenting effect which is far more useful. 

In [69]:
class NameTooShortError(ValueError):
    pass

def validate(name):
    if len(name) < 10:
        raise NameTooShortError(name)

In [70]:
validate('Trent')

NameTooShortError: Trent

We can actually push this up a level of abstraction, defining a base 'error' class from which other error classes inherit. 

In [73]:
class BaseValidationError(ValueError):
    pass

class NameTooShortError(BaseValidationError):
    pass

class NameTooLongError(BaseValidationError):
    pass

class NameTooCuteError(BaseValidationError):
    pass

name = 'Trent'
try: 
    validate(name)
except BaseValidationError as err:
    NameTooShortError(err)

#### Cloning Objects for Fun and Profit. 

In python assignment statements don't create copies, they bind names to objects. With immutable data structures we don't usually care, but it's sometimes helpful to be able to create an actual copy to work with independently of the original. 

One way we could create a copy of mutable collections like lists, dicts, and sets is by passing an existing list, dict, or set into the appropriate factory function and then reassigning it:

`new_list = list(old_list)` 

But this technique creates only a shallow copy, and it is important to understand the distinction between deep and shallow copies. 

A shallow copy makes a new collection object and then populates it with *references* to the items contained in the original, not actual *copies* of those items. A deep copy on the other hand, crawls through the original collection object making clones of everything it finds and placing those clones in the new object. 

Put another way, shallow copies don't recurse while deep copies do. 

In [5]:
# Making a shallow copy.

xs = [[1,2,3], [4,5,6], [7,8,9]]
ys = list(xs)

# These two objects contain the same elements, but watch what happens when 
# we add a new sublist.

xs.append(['A new sublist'])
xs, ys

([[1, 2, 3], [4, 5, 6], [7, 8, 9], ['A new sublist']],
 [[1, 2, 3], [4, 5, 6], [7, 8, 9]])

If we change any of the *original* elements in xs, the change will be reflected in ys:

In [4]:
xs[1][0] = 'X'
xs, ys

([[1, 2, 3], ['X', 5, 6], [7, 8, 9], ['A new sublist']],
 [[1, 2, 3], ['X', 5, 6], [7, 8, 9]])

The questions which remain now are: how do we create deep copies, and how do create copies of non-standard objects like custom classes?

The answer: the Python `copy` module.

In [9]:
import copy 
xs = [[1,2,3],[4,5,6],[7,8,9]]
zs = copy.deepcopy(xs)

xs[1][0] = 'X'

print(xs,'\n', zs)

[[1, 2, 3], ['X', 5, 6], [7, 8, 9]] 
 [[1, 2, 3], [4, 5, 6], [7, 8, 9]]


Making shallow or deep copies of custom objects is done through the `copy` class again. `copy.copy()` for shallow copies, `copy.deepcopy()` for deeper ones. 

In [13]:
class Point:
    def __init__(self, x, y,):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Point({self.x!r}, {self.y!r})'
    
a = Point(23,42)
b = copy.copy(a)

print(a is b)

False


#### Abstract Base Classes Keep Inheritance in Check

Abstract base classes are good for insuring that derived classes implement specific methods from the base class. Ideally we want two things from an abstract base class: we want it to be impossible to implement the base class, we want any lapses in creating interfaces for subclasses errors out as soon as possible. Here's an example of this not being done well:

In [18]:
class Base:
    def foo(self):
        raise NotImplementedError()
        
    def bar(self):
        raise NotImplementedError()
        
class Concrete(Base):
    def foo(self):
        return 'foo implemented'
    
# Here we instantiate the base class, which is a no-no.
b = Base()
b.bar()

NotImplementedError: 

In [19]:
# Here we instantiate Concrete, but it doesn't throw an error until we call the wrong method. 

c = Concrete()
c.foo()

'foo implemented'

In [20]:
c.bar()

NotImplementedError: 

Since Python 2.6, the abc module has given us tools for better handling this issue.

In [24]:
from abc import ABCMeta, abstractmethod

class Base(metaclass=ABCMeta):
    
    @abstractmethod
    def foo(self):
        pass
    
    @abstractmethod
    def bar(self):
        pass
    
class Concrete(Base):
    def foo(self):
        pass
    

b = Base()

TypeError: Can't instantiate abstract class Base with abstract methods bar, foo

In [25]:
c = Concrete()

TypeError: Can't instantiate abstract class Concrete with abstract methods bar

#### What NamedTuples are good for

We already have standard tuples, so what do we need an additional tuple data structure for? 

Tuples have a couple of downsides. First, there's no way to attach names to individual elements within a tuple, so the only way to get anything out is through indexing. Second, it's pretty hard to insure that two tuples have the same content. 

NamedTuples solve these two problems, while still being immutable data containers just like tuples. Data stored in a top-level attribute can't be changed, but all data can be accessed through a human-readable name. 

In [2]:
from collections import namedtuple

Car = namedtuple('Car', 'color mileage')

Here we're using the namedtuple factory function to create what's basically a class. The first argument is a `typename` argument which tells namedtuple the name of the variable to which we're assigning it (otherwise it has no way of knowing). The next argument is a single string containing names for whatever we'll eventually include in `Car`, on which namedtuple calls `.split()` to create a list of strings (though it is also syntactically correct to pass in a list of strings at this point). 

In [3]:
my_car = Car('green', 1234)
my_car.color

'green'

In [4]:
my_car.mileage

1234

In [5]:
# But integer indexing also works, so NamedTuples can be dropped in as replacements for tuples.

my_car[0]

'green'

In [6]:
# We can also tuple unpack it
print(*my_car)

green 1234


Internally a namedtuple is just an immutable class that's a little more memory efficient. 

#### Subclassing named tuples

Though there may not be a way to do it in the original named tuple definition, with subclassing we can create named tuples which have methods. 

In [9]:
Student = namedtuple('Student', 'age weight iq')

class StudentWithMethods(Student):
    def student_greeting(self):
        print('Hello')
        
stud_2 = StudentWithMethods(13, 100, 110)
stud_2.student_greeting()

Hello


It's even possible to use a kind of inheritance to add immutable fields to a new namedtuple 

In [13]:
ElectricCar = namedtuple('ElectricCar', Car._fields + ('charge',))

In [14]:
new_electric = ElectricCar('green', 40000, 40)

Some useful `namedtuple` helper functions include `._asdict()`, `._replace()`, and `._make()`. 

In [17]:
new_electric._asdict()

OrderedDict([('color', 'green'), ('mileage', 40000), ('charge', 40)])

In [21]:
new_new_electric = new_electric._replace(color='blue')
new_new_electric.color

'blue'

#### Class v.s. Instance Pitfalls

Just as Python's object model distinguishes between class methods and instance methods, it also distinguishes between class variables and instance variables. Class variables are stored in the class definition and as such are not tied to any particular instance of a class. When they are changed all class instances have access to the new values. This is not the case for instance variables, which are specific to their instance. 


In [22]:
class Dog:
    num_legs = 4 # class variable 
    
    def __init__(self, name):
        self.name = name # defined within the class, but will be an instance variable. 
        


`num_legs` is accessible through instances or through the class itself. 

In [23]:
Dog.num_legs

4

In [24]:
jack = Dog('jack')
jack.num_legs

4

In [25]:
# Look at what happens here:

Dog.name

AttributeError: type object 'Dog' has no attribute 'name'

Though it was defined in the `Dog` definition it is unique to each instance and therefore not accessible through `Dog` as a class. 

The class variable `num_legs` can be overridden for specific instances:

In [29]:
jill = Dog('jill')

jack.num_legs = 6

print(jill.num_legs, jack.num_legs, jack.__class__.num_legs)

4 6 4


This can lead to confusion, as we've overwritten the class variable in the `jack` instance.

On the other hand, we can do some pretty nifty stuff with these class variables. 

In [36]:
class CountObject:
    num_instances = 0
    
    def __init__(self):
        self.__class__.num_instances += 1
        
ob1 = CountObject()
print(ob1.num_instances)

1


In [37]:
ob2 = CountObject()
print(ob2.num_instances)

2


In [38]:
# But check this out: 

ob1.num_instances

2

Without the `self.__class__.num_instances += 1` syntax above, we aren't incrementing the shared class variable, only the instance variable. For each instance we are correctly incrementing by one unit but aren't changing the class variable. 

In [39]:
class badCountObject:
    num_instances = 0
    
    def __init__(self):
        self.num_instances += 1
        
badob1 = badCountObject()
badob1.num_instances

1

In [40]:
badob2 = badCountObject()
badob2.num_instances

1

#### Instance, Class, and Static Methods Demystified 

In [46]:
class MyClass:
    
    def instance_method(self):
        return "Instance method called", self
    
    @classmethod
    def class_method(cls):
        return "Class method called", cls
    
    @staticmethod
    def static_method():
        return "Static method called"
    
ex_cls = MyClass()
ex_cls.static_method()

'Static method called'

The `instance_method` is exactly what it sounds like. It takes the `self` parameter and uses this to manipulate the state of the instance it's a part of. But it can also change class state by accessing the class directly via the `self.__class__` attribute.

The `class_method` is marked by the `@classmethod` decorator and takes a `cls` argument instead of `self`. This way it points directly to the class, not the instance. 

The `static_method` is marked by the `@staticmethod` decorator. It takes no arguments and is somewhat restricted in the data that it can access. 

In order to better understand what's actually occurring here it'll be useful to actually call these methods. The instance method and class method return a tuple which'll help us trace the process. 

In [48]:
obj1 = MyClass()
obj1.instance_method()

('Instance method called', <__main__.MyClass at 0x106634588>)

In [49]:
obj1.class_method()

('Class method called', __main__.MyClass)

In [50]:
obj1.static_method()

'Static method called'

The `obj1.instance_method()` is what's known as syntactic sugar, which are Python affordances that make our lives easier. We could also pass in the `obj1` directly to evoke the same response:

In [51]:
MyClass.instance_method(obj1)

('Instance method called', <__main__.MyClass at 0x106634588>)

If you're puzzled by the fact that the instance method returns `<__main__.MyClass at 0x106634588>` while the class method returns a different `__main__.MyClass`, this is because the default `__repr__` attribute is different for classes and instances:

In [52]:
obj1.__repr__

<method-wrapper '__repr__' of MyClass object at 0x106634588>

In [53]:
MyClass.__repr__

<slot wrapper '__repr__' of 'object' objects>

As far as I know this difference isn't substantive. Now for the static method:

In [54]:
obj1.static_method()

'Static method called'

In [55]:
MyClass.static_method()

'Static method called'

Creating a static method is basically a way of putting a method name into both the class's and all subsequent instance's respective namespaces -- that is, making it accessible. It has restrictions on its behavior enforced under the hood by the fact that Python doesn't pass in `self` or `cls` arguments, which otherwise would allow it to manipulate instance and class states. 